# Tarea 2
A partir del corpus proporcionado realizar un modelo del lenguaje neuronal con base en la arquitectura de word2vec. Siganse los siguientes pasos:

1. Limpiar los Textos y apliccar stemming a las palabras. **[DONE]**
2. Insertar simbolos de inicio y final de cadena. **[DONE]**
3. Obtener los bigramas que aparecen en este texto. **[DONE]**
4. Entrenar con los bigramas la red neuronal y obtener los valores para los hiperparamteros. Tomar de 100 a 300 unidades para la capa oculta.
5. Obtener la matriz A y Pi a partir de las salidas de la red neuronal 
6. Calcular la probabilidad de las siguientes oraciones:
    - Nos bañamos con agua caliente 
    - El animalito le olía la cabeza
    - Pascuala ordeñaba las vacas

In [1]:
# Importamos las librerias

# Usamos os para cargar los libros
from os import listdir,getcwd
from os.path import isfile, join
import re


## Importamos el corpus
Lo primero que hacemos es importar el corpus al notebook para que podamos utilizarlos. En este caso definimos dos formas de cargar el corpus, ya sea por documento o cargando todos los documentos del folder.

In [2]:

# Obtenemos el path del folder donde se almacenan los corpus
folder_path = (getcwd() + r"/CorpusDocs")

# Los almacenamos en una lista donde se almacenan los nombres de los archivos.
# Esto es en caso de que usemos todos los corpus.
corpus_name_list = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]

def loadAllCorpus():
    """
    Esta funcion carga todos los corpus que estan en el folder Corpus Docs.
    """
    corpis = ''
    for file in corpus_name_list:
        with open("./CorpusDocs/" + file, 'r', encoding="utf8") as f:
            corpis += f.read()
    return corpis

def loadCorpus(corpus_name):
    """
    Esta funcion nos sirve para cargar un corpus especifico
    """
    with open("./CorpusDocs/" + corpus_name, 'r', encoding="utf8") as f:
        corpus = f.read()
    return corpus
        

In [3]:
# Cargamos el corpus.

#corpus = loadAllCorpus()
corpus = loadCorpus('corpusML.txt')

## Limpieza del Texto
Separamos las palabras de las oraciones para poder trabajar con ellas individualmente

In [4]:
def add_eos_init(corpus):
    """
    Adds <eos> and <init> to a corpus. Based on line jumps
    """
    
    init = '<init> '
    eos = ' <eos>'
    corpus_in_eo = []


    corpus_in_eo = init + corpus + eos
    corpus_in_eo= corpus_in_eo.replace("\n", eos +" \n"+init)

    return corpus_in_eo

In [5]:
corpus_init = add_eos_init(corpus)
words = corpus_init.split()
print(words[:20])

['<init>', 'Comencé', 'a', 'trabajar', 'y', 'me', 'pegaron,', 'me', 'maltrataron', 'con', 'chicote', '<eos>', '<init>', 'Mis', 'patrones', 'me', 'pegaron', 'porque', 'no', 'me']


Eliminamos la puntuación del documento, acentos y normalizamos el texto en minusculas. Para hacer la eliminación de los símbolos de puntuación utilziamos una tabla de traducción para optimizar la velocidad de procesamiento. Tambien fue necesario extender la tabla de símbolos para que incluyera algunos símbolos latinos que faltaban.

Para eliminar acentos usamos la libreria unidecode que se tiene que instalar adicionalmente: `pip install unidecode`

In [6]:
import string
import unidecode

# Para poder mantener algunos flags quitamos < > de los elementos que se pueden eliminar
# de los simbolos de puntuación
punct = string.punctuation.replace("<", '')
punct = punct.replace(">", '')

print(punct)

!"#$%&'()*+,-./:;=?@[\]^_`{|}~


In [7]:
lat_punctuation = punct+'¿¡1234567890'
#print(lat_punctuation)

table = str.maketrans('', '', lat_punctuation)

In [8]:
clean_words = []

for word in words:
    word = word.lower()               # Minusculas
    word = unidecode.unidecode(word)  # Quitamos acentos.
    
    # Clean punctuation
    temp_w = []
    for letter in word:
        if letter not in lat_punctuation:
            temp_w.append(letter)
        word = ''.join(temp_w)

    clean_words.append(word)


## Stemming de Palabras
Para hacer el stemming de las palabras usamos NLTK. Para esto hay que instalar NLTK: 

Lo primero que hacemos es definir un stemmer. En este caso usaremos [Snowball Stemmer](http://snowball.tartarus.org/texts/introduction.html).

In [9]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

In [10]:
stemmed_text = []
for word in clean_words:
    stemmed_text.append(stemmer.stem(word))
    
print(stemmed_text[:10])

['<init>', 'comenc', 'a', 'trabaj', 'y', 'me', 'peg', 'me', 'maltrat', 'con']


## Bigramas
Para obtener los biogramas existentes del corpus creamos una fucnion que nos sierve para obtener todos los bigramas que existen en el corpus y a traves de el generar una lista de ellos.

In [11]:
def create_ngrams(stemmed_text, n):
    """
    Creates an n-gram structure from a stemmed or tokenized text. 
    
    Params
    ------
    stemmed_text: Tokens or stemmed words of a corpus
    n: the size of the n gram ex. 2 for a bigram
    """
    return zip(*[stemmed_text[i:] for i in range(n)])

In [12]:
bigramas = list(create_ngrams(stemmed_text, 2))

In [13]:
print(bigramas[0])

('<init>', 'comenc')


In [14]:
import collections
counter=collections.Counter(bigramas)       
    

In [15]:
print(counter)

Counter({('<eos>', '<init>'): 1704, ('<init>', 'pues'): 177, ('entonc', '<eos>'): 86, ('<init>', 'se'): 79, ('<init>', 'me'): 77, ('otra', 'vez'): 72, ('<init>', 'y'): 68, ('<init>', 'per'): 65, ('<init>', 'no'): 61, ('<init>', 'entonc'): 49, ('<init>', 'despu'): 47, ('nom', '<eos>'): 44, ('<init>', 'el'): 43, ('<init>', 'lueg'): 43, ('par', 'que'): 38, ('cas', '<eos>'): 35, ('<init>', 'cuand'): 33, ('<init>', 'si'): 31, ('en', 'la'): 30, ('<init>', 'com'): 30, ('<init>', 'este'): 29, ('a', 'la'): 27, ('pues', 'se'): 26, ('<init>', 'le'): 26, ('pues', 'me'): 26, ('<init>', 'lo'): 26, ('pues', 'entonc'): 26, ('<init>', 'ya'): 25, ('en', 'el'): 25, ('<init>', 'la'): 25, ('<init>', 'por'): 25, ('com', '<eos>'): 24, ('per', 'pues'): 23, ('que', 'se'): 21, ('no', 'se'): 21, ('que', 'no'): 21, ('fue', 'a'): 21, ('trabaj', '<eos>'): 20, ('<init>', 'habi'): 20, ('por', 'eso'): 20, ('iba', 'a'): 20, ('ya', 'no'): 19, ('con', 'el'): 19, ('<init>', 'he'): 19, ('y', 'pues'): 18, ('de', 'la'): 18, 

In [16]:
print(counter['me', 'peg'])

15


In [17]:
# Obtenemos el alfabeto de las palabras del stem
#alfabeto = set(stemmed_text)

alfabetoPI = []
for stem in stemmed_text:
    if stem not in alfabetoPI:
        alfabetoPI.append(stem)
        

alfabetoPI.remove('<init>')

print("AlfabetoPI total: {0}".format(len(alfabetoPI)))


AlfabetoPI total: 1216


## Visualización de Matrices
Utilizamos Pandas para visualizar la matriz A y la matriz Pi

In [18]:
import pandas as pd

In [19]:
MatrixA = pd.DataFrame(index=alfabetoPI, columns=alfabetoPI)

In [20]:
for x in alfabetoPI:
    for y in alfabetoPI:
        MatrixA.set_value(y,x, counter[y,x]+1)

d:\python 3.5\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
MatrixA

,comenc,a,trabaj,y,me,peg,maltrat,con,chicot,<eos>,...,juguetit,entram,cab,bebecit,tabiqu,calent,pajuel,vapor,quemart,cai
comenc,1,2,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
a,1,1,15,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
trabaj,1,3,1,4,1,1,1,1,1,21,...,1,1,1,1,1,1,1,1,1,1
y,1,9,2,1,13,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
me,1,1,1,1,1,16,3,1,1,1,...,1,1,1,1,1,1,1,1,1,2
peg,1,6,1,1,2,1,1,2,1,12,...,1,1,1,1,1,1,1,1,1,1
maltrat,1,1,1,1,1,1,1,2,1,3,...,1,1,1,1,1,1,1,1,1,1
con,1,1,5,1,1,1,1,1,2,1,...,1,1,1,1,2,1,1,1,1,1
chicot,1,1,1,1,1,1,1,1,1,2,...,1,1,1,1,1,1,1,1,1,1
<eos>,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [22]:
MatrixPI = pd.DataFrame(index=alfabetoPI, columns=['<init>'])

In [23]:
for y in alfabetoPI:
    if y is not '<eos>':
        MatrixPI.set_value(y,'<init>', counter['<init>',y])

d:\python 3.5\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
MatrixPI

,<init>
comenc,1
a,6
trabaj,2
y,68
me,77
peg,0
maltrat,0
con,9
chicot,0
<eos>,1


# One Hot

In [25]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(alfabetoPI)
print(integer_encoded)

[ 217    2 1108 ... 1155  939  147]


In [26]:
Matrix_One_Hot = pd.DataFrame(index=integer_encoded, columns=integer_encoded)
Values_MatrixA = MatrixA.get_values()
w = 0
j = 0
for x in integer_encoded:
    for y in integer_encoded:
        Matrix_One_Hot.set_value(y,x,Values_MatrixA[j,w])
        j = j + 1
    j=0
    w = w + 1
Matrix_One_Hot

d:\python 3.5\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


,217,2,1108,1207,686,842,660,227,191,1,...,593,426,140,118,1056,150,808,1155,939,147
217,1,2,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,15,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1108,1,3,1,4,1,1,1,1,1,21,...,1,1,1,1,1,1,1,1,1,1
1207,1,9,2,1,13,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
686,1,1,1,1,1,16,3,1,1,1,...,1,1,1,1,1,1,1,1,1,2
842,1,6,1,1,2,1,1,2,1,12,...,1,1,1,1,1,1,1,1,1,1
660,1,1,1,1,1,1,1,2,1,3,...,1,1,1,1,1,1,1,1,1,1
227,1,1,5,1,1,1,1,1,2,1,...,1,1,1,1,2,1,1,1,1,1
191,1,1,1,1,1,1,1,1,1,2,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


# Red Neuronal
## N-300-N

In [91]:
import random
import numpy as np
import math

def softmax(x,vx):
    vy = np.zeros([len(vx)])
    for i in range(0,len(vy)-1):
        vy[i] = math.exp(vx[i])
    sf = math.exp(x) / sum(vy)
    return sf

def softmaxP(phi,y):
    return phi-y

def obtainTarget():
    values = np.zeros([1216])
    for i in range(0,1215):
        values[i] = Matrix_One_Hot.columns[i]
    return values

def EntropyX(y,p):
    return (1/softmaxP(y,p)) * math.exp(p)

def Dout(phi,y):
    x = softmaxP(phi,y)
    if x == 0:
        return 1
    else:
        return 0

#Revisar función de riesgo
def riego(y,p):
    return -sum(y)*math.log(EntropyX(y,p))
    
#Funciones para la red neuronal
def weight_B_Init():
    bias = random.random()
    weight = random.random()
    return weight, bias
    
def randomProbability():
    wX = np.zeros([300])
    bX = np.zeros([300])
    for i in range(0,299):
        wX[i], bX[i] = weight_B_Init()
    return wX,bX

def lineal(x):
    if(x<0):
        return 0
    else:
        return x
def error(x,y):
    return y-x

def feedForward(bigrama,probabilityWhen,v_W,v_B,v_P,j):
    #Rango de aprendizaje
    alpha = 0.01
    a0 = probabilityWhen
    a1 = lineal(v_W*a0 + v_B)
    
    u = probabilityWhen*a1 + v_B
    a2 = softmax(u,v_P)
    
    e = error(a2,bigrama[1])
    s2 = (-2)*softmaxP(a1,probabilityWhen) * e
    
    u = np.array([[softmax(a1,v_P),0]])
    s1 = u * probabilityWhen * s2
    
    w2_1 = probabilityWhen - alpha*s2*a1
    b2_1 = v_B - alpha*s2
    
    v_P[j] = alpha * EntropyX(s2,probabilityWhen)
#    print(EntropyX(Total_Value,probabilityWhen))
    return w2_1, b2_1

# Creación de la red neuronal

In [90]:
#Se crean 300 pesos y bias aleatorios
vector_Weight, vector_Bias = randomProbability()

#Matriz de salida(probabilidad de cada bigrama bigrama)
matrix_Probability = Matrix_One_Hot / 1216 #Se inicializa

# Variables para movernos sobre la tabla 
count_Context = 0
count_Target = 0


################# EJEMPLO PARA EL CONTEXTO 1 ########################

#Tomamos el contexto X
bigram_Context = Matrix_One_Hot.columns[count_Context]
#Tomamos el vector con todas las posibles combinaciones
bigram_Target = obtainTarget()

#Tomamos la probabilidad de la columna
vector_Probability = matrix_Probability.iloc[count_Context,:].get_values()
#Se calcula la probabilidad total del vector_Probability
total_Vector_Probability = sum(vector_Probability)
#Ahora ingresamos las posibles combinaciones del bigrama a la red neuronal

for j in range(0,1215): #Primer Ciclo -> Recorrer todos los bigramas de un solo contexto
    #Ahora calculamos la probabilidad del bigrama entre la sumatoria de todas las combinaciones del contexto
    probabilityWhen =  matrix_Probability.iloc[count_Context,count_Target] / total_Vector_Probability
    for i in range(0,299): #Segundo Ciclo -> Pasar por los 300 nodos
        bigram = [bigram_Context, bigram_Target[count_Target]] #Se crea el bigrama (Contexto,Target)
        vector_Weight[i], vector_Bias[i]= feedForward(bigram,probabilityWhen,
                                                      vector_Weight[i],vector_Bias[i],
                                                      vector_Probability,j)
    i = 0
    count_Target = count_Target + 1

vector_Probability

KeyboardInterrupt: 

# Realizado por:
- Bustos Ramírez Luis Enrique
- Flores Cortés Juan Pablo
